# Finetune Masked-AutoEncoder

- Finetune the deep network after pretraining the self-supervised learning framework.

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%cd ..

D:\GitHub\eeg_analysis


In [2]:
# Load some packages
import os
import gc
from copy import deepcopy
import hydra
from omegaconf import OmegaConf
import wandb
import pprint
import numpy as np
import torch
from tqdm.auto import tqdm
from collections import OrderedDict

import matplotlib
import matplotlib.pyplot as plt
import scienceplots

# custom package
from run_train import check_device_env
from run_train import set_seed
from run_train import compose_dataset
from run_train import generate_model
from train.ssl_train_script import ssl_train_script
from train.train_script import train_script
from models.utils import count_parameters

---

## Specify the dataset, model, and train setting

In [3]:
pre_model_path = 'local/checkpoint/'
pre_model_name = '2ew55ua4'

use_wandb = False
project = 'caueeg-mae'
device = 'cuda:0'

In [4]:
print('PyTorch version:', torch.__version__)
device = torch.device(device if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.')

PyTorch version: 2.0.0+cu117
cuda is available.


---

## Load and modify the pretrained network

In [5]:
# load pretrained configurations
path = os.path.join(pre_model_path, pre_model_name.split(',')[-1], 'checkpoint.pt')
try:
    ckpt = torch.load(path, map_location=device)
    config = ckpt['config']
except Exception as e:
    print(e)
    print(f'- checkpoint cannot be opened: {path}')
pprint.pprint(config)

{'EKG': 'X',
 '_target_': 'models.ssl.mae.mae_pre_b_e768_d512',
 'activation': 'gelu',
 'age_mean': tensor([71.1417], device='cuda:0'),
 'age_std': tensor([9.7264], device='cuda:0'),
 'awgn': 0.01,
 'awgn_age': 0.001,
 'base_lr': 0.00012226453861270155,
 'class_label_to_name': ['Normal', 'MCI', 'Dementia'],
 'class_name_to_label': {'Dementia': 2, 'MCI': 1, 'Normal': 0},
 'criterion': 'cross-entropy',
 'crop_length': 4096,
 'crop_multiple': 4,
 'crop_timing_analysis': False,
 'cwd': 'D:\\GitHub\\eeg_analysis',
 'dataset_name': 'CAUEEG dataset',
 'dataset_path': 'local/dataset/caueeg-dataset/',
 'ddp': False,
 'device': device(type='cuda'),
 'draw_result': True,
 'file_format': 'memmap',
 'in_channels': 19,
 'input_norm': 'datapoint',
 'iterations': 390625,
 'latency': 2000,
 'load_event': False,
 'lr_scheduler_type': 'cosine_decay_with_warmup_half',
 'mask_ratio': 0.5,
 'mgn': 0.01,
 'minibatch': 256,
 'minibatch_3090': 256,
 'mixup': 0.0,
 'multi_batch_size': 32,
 'norm_pix_loss': Fals

## Finetuning

In [ ]:
pre_model = deepcopy(model)
pre_model_state = pre_model.state_dict()

In [ ]:
# training configuration
config['project'] = project
config['use_wandb'] = use_wandb
config['pre_model'] = pre_model_name
config['device'] = device

config['total_samples'] = 5.0e+5
config['search_lr'] = False
config['base_lr'] = 1e-3
config['lr_scheduler_type'] = 'cosine_decay_with_warmup_half'

config["warmup_min"] = 200   

# model
tuning_type = "finetune"  # "finetune", "fc_stage", "layer-wise finetune"

config["out_dims"] = 3
config["task"] = "dementia"
config["use_age"] = 'fc'
config["fc_stages"] = 3
config["global_pool"] = True
config["dropout"] = 0.3

In [ ]:
# generate the model
config["_target_"] = config["_target_"].replace('.ssl', '').replace('_pre', '')
model = generate_model(config).to(device)

# load the model
model_state = model.state_dict()
for k, v in model_state.items():
    if not k.startswith('fc'):
        model_state[k] = pre_model_state[k]

model.load_state_dict(model_state)

In [ ]:
if tuning_type == "fc_stage":
    model.requires_grad_(False)
    model = model.eval()    
    model.fc_stage.requires_grad_(True)
    model.fc_stage.train()
elif tuning_type == "finetune":
    model.requires_grad_(True)
    model = model.train()    
    model.art_net.requires_grad_(False)
    model.art_net = model.art_net.eval()
elif tuning_type == "layer-wise finetune":
    model.requires_grad_(True)
    model = model.train()    
    model.art_net.requires_grad_(False)
    model.art_net = model.art_net.eval()

config["num_params"] = count_parameters(model)

# print(model)

In [ ]:
for name, param in model.named_parameters():
    print(f"{name:100}\t|\t{param.requires_grad}")

In [ ]:
# check the workstation environment and update some configurations
check_device_env(config)

# collect some garbage
gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()

# fix the seed for reproducibility (a negative seed value means not fixing)
set_seed(config, rank=None)

# compose dataset
train_loader, val_loader, test_loader, multicrop_test_loader = compose_dataset(config)

pprint.pprint(config)

In [ ]:
# train
train_script(
    config,
    model,
    train_loader,
    val_loader,
    test_loader,
    multicrop_test_loader,
    config["preprocess_train"],
    config["preprocess_test"],
)